In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import time
import pydde as d

In [2]:
#Parameters
samplenum = 10
epochs = 20
input_size = 3
output_size = 3
learning_rate = 0.01
time_length = 3; #seconds
hiddenlayers = [90]

In [3]:
# Generate simulation
dyn = d.PyDyn('../Data/point-mass_pendulum.sim', time_length)
state_init = dyn.compute(dyn.p_init)
f = dyn.f(state_init, dyn.p_init)
df = dyn.df_dp(state_init, dyn.p_init)
dy = dyn.dy_dp(state_init, dyn.p_init)

#Sample targets only variables in z direction
y_target = np.zeros((samplenum, 3))
y_target[:,2] = np.random.rand(samplenum)
y_target[:,1] = 2
print(y_target)
p = np.ones((3*time_length, samplenum))
for i in range(samplenum):
    p[:,i] = dyn.get_p(y_target.transpose()[:,i], dyn.p_init)
print(p.shape)

[[0.         2.         0.2207534 ]
 [0.         2.         0.27073957]
 [0.         2.         0.11730403]
 [0.         2.         0.74790176]
 [0.         2.         0.90423131]
 [0.         2.         0.51426679]
 [0.         2.         0.4208993 ]
 [0.         2.         0.26617991]
 [0.         2.         0.32218653]
 [0.         2.         0.87319777]]
(9, 10)


## Building the custon Simulation Activation Function

In [6]:
class Simulate(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        #print(f'input: {input.shape}')
        p = input.clone().numpy().transpose()
        y_pred = torch.ones([samplenum,3])
        for i in range(len(p[0, :])):
            state = dyn.compute(p[:,i])
            y_pred[i, :] = torch.tensor(state.y[-3:])
        #print(f'y_pred: {y_pred.shape}')
        
        ctx.save_for_backward(input)
        
        return y_pred
        
    @staticmethod
    def backward(ctx, grad_output):
        print(grad_output)
        input, = ctx.saved_tensors
        p = input.clone().numpy().transpose()
        dy_dp_batch = torch.zeros([3, 3*time_length])
        for i in range(samplenum):
            state= dyn.compute(p[:, i])
            dy_dp = dyn.dy_dp(state, p[:, i])
            dy_dp = torch.tensor(dy_dp[-3:, :])
            dy_dp_batch = dy_dp_batch + dy_dp
        print(f'shape of dy/dp_batch: {dy_dp_batch.shape}')
        
        grad_input = torch.tensor(dy_dp_batch/samplenum).t().mm(grad_output.t().double()).t()
        #print(f'shape of grad input: {grad_input.shape}')
        #print(f'shape of grad output: {grad_output.shape}')
        return grad_input

Simulate = Simulate.apply

## FE Test for custom function

In [9]:
# Error for whole simulation
from numpy import linalg as LA

FE = 1e-6
p = torch.tensor(p, requires_grad = True).double()
y = Simulate(p.t())
print(y.shape)
#test = sum(y)
#test.backward()
grad_output = torch.ones([10,3]).double()
y.backward(grad_output)
dy_dp = p.grad.double()
print(f'dy_dp shape = {p.grad.shape}')
print(f'dy_dp = {dy_dp}')

torch.Size([10, 3])
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
shape of dy/dp_batch: torch.Size([3, 9])
dy_dp shape = torch.Size([9, 10])
dy_dp = tensor([[-2.9386e-04, -2.9386e-04, -2.9386e-04, -2.9386e-04, -2.9386e-04,
         -2.9386e-04, -2.9386e-04, -2.9386e-04, -2.9386e-04, -2.9386e-04],
        [-9.1943e-02, -9.1943e-02, -9.1943e-02, -9.1943e-02, -9.1943e-02,
         -9.1943e-02, -9.1943e-02, -9.1943e-02, -9.1943e-02, -9.1943e-02],
        [ 1.5787e-04,  1.5787e-04,  1.5787e-04,  1.5787e-04,  1.5787e-04,
          1.5787e-04,  1.5787e-04,  1.5787e-04,  1.5787e-04,  1.5787e-04],
        [ 1.1461e-02,  1.1461e-02,  1.1461e-02,  1.1461e-02,  1.1461e-02,
          1.1461e-02,  1.1461e-02,  1.1461e-02,  1.1461e-02,  1.1461e-02],
        [ 1.6780e-01,  1.6780e-01,  1.6780e-01,  1.6780e-01,  1.6780e-01,
      

In [ ]:
dy_dp_FD = np.zeros((samplenum,len(dyn.p_init)))
#dy_dp_FD = np.zeros((3,len(dyn.p_init)))

for i in range(len(dyn.p_init)):
    dp= np.zeros(len(dyn.p_init))
    dp[i] = FE
    dp = torch.tensor(dp)
    y_p = Simulate(p + dp)
    y_m = Simulate(p - dp)
    y_p = y_p.detach().numpy()
    y_m = y_m.detach().numpy()
    dy_dp_FD[0, i] = sum((y_p - y_m) / (2* FE))
    #dy_dp_FD[:, i] = (y_p - y_m) / (2* FE)


print(f' dy_dp_FD = {dy_dp_FD}')
print(f' dy_dp_FD = {dy_dp_FD.shape}')
dy_dp = dy_dp.detach().numpy()
err = LA.norm(dy_dp_FD - dy_dp)
print(err)

## FD Test for dy_dp Function

In [16]:
from numpy import linalg as LA

#Calculate dy_dp with FE
FE = 1e-6
dy_dp = dyn.dy_dp(state_init, dyn.p_init)
dy_dp_FD = np.zeros((len(dyn.p_init),len(dyn.p_init)))

for i in range(len(dyn.p_init)):
    dp= np.zeros(len(dyn.p_init))
    dp[i] = FE
    pos = dyn.p_init + dp
    y_p = dyn.compute(pos)
    y_m = dyn.compute(dyn.p_init - dp)
    dy_dp_FD[:, i] = (y_p.y - y_m.y) / (2* FE)
err = LA.norm(dy_dp_FD - dy_dp)
print(err)


2.5942135105905507e-06
